In [1]:
import io

import pyterrier as pt
from experiment_utils.experiments_helper import get_timeit_dependencies_name, getOptimalAlpha, latency_per_query, \
    load_dense_index_from_disk
from latency.latency_utils import preprocess_data, get_vector_ids

if not pt.started():
    pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
from encoders.bge_base_en import BgeQueryEncoder

package = "BAAI/"
model_name = "bge-base-en-v1.5"
q_encoder = BgeQueryEncoder(package + model_name)
model_directory = 'bge'
path_to_root = "../../"
pipeline_name = "BM25 >> bge-base-en-v1.5"

In [3]:
prefix = "irds:beir/"
test_suffix = "/test"

In [4]:
def get_timeit_dep(dataset_name):
    results_lexical_retriever, semantic_reranker = get_timeit_dependencies_name(prefix + dataset_name,
                                                                                prefix + dataset_name + test_suffix,
                                                                                q_encoder,
                                                                                model_name,
                                                                                path_to_root, model_directory,
                                                                                alpha=getOptimalAlpha(
                                                                                    prefix + dataset_name,
                                                                                    pipeline_name, model_directory))
    dense_index = load_dense_index_from_disk(prefix + dataset_name, q_encoder,
                                             model_name,
                                             path_to_root, model_directory)

    processed_df, queries = preprocess_data(results_lexical_retriever)

    return processed_df, queries, dense_index, results_lexical_retriever, semantic_reranker

In [5]:
from contextlib import redirect_stdout


def measure_latency(dataset_name):
    processed_df, queries, dense_index, results_lexical_retriever, semantic_reranker = get_timeit_dep(dataset_name)

    f = io.StringIO()
    with redirect_stdout(f):
        %timeit dense_index.encode_queries(queries)
    timeit_output = f.getvalue()

    result = latency_per_query(timeit_output, prefix + dataset_name, test_suffix, pipeline_name, model_directory,
                               "query_encoding_")
    print("Query encoding time: " + result)

    vector_ids = get_vector_ids(processed_df)

    f = io.StringIO()
    with redirect_stdout(f):
        %timeit dense_index.get_current_vectors(vector_ids)
    timeit_output = f.getvalue()

    result = latency_per_query(timeit_output, prefix + dataset_name, test_suffix, pipeline_name, model_directory,
                               "vector_embedding_")
    print("Document retrieval time: " + result)


In [6]:
import traceback


def run_latency_datasets():
    for dataset_name in dataset_list:
        try:
            measure_latency(dataset_name)
        except Exception as e:
            traceback.print_exc()

In [7]:
dataset_list = ["fiqa", "nfcorpus", "scifact"]

In [8]:
run_latency_datasets()

100%|██████████| 57638/57638 [00:00<00:00, 1552414.46it/s]


16 s +- 599 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 24.6914 ms. Experiment details: 16 s +- 599 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

158 ms +- 16.5 ms per loop (mean +- std. dev. of 7 runs, 10 loops each)

Document retrieval time: Latency per query: 0.2438 ms. Experiment details: 158 ms +- 16.5 ms per loop (mean +- std. dev. of 7 runs, 10 loops each)



100%|██████████| 3633/3633 [00:00<00:00, 1554729.77it/s]


4.59 s +- 52.7 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 14.2105 ms. Experiment details: 4.59 s +- 52.7 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

6.17 ms +- 15.6 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0191 ms. Experiment details: 6.17 ms +- 15.6 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



100%|██████████| 5183/5183 [00:00<00:00, 1610898.68it/s]


10.4 s +- 156 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 34.6667 ms. Experiment details: 10.4 s +- 156 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

9.6 ms +- 86.4 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.032 ms. Experiment details: 9.6 ms +- 86.4 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



In [9]:
model_name = "bge-small-en-v1.5"
pipeline_name = "BM25 >> " + model_name
q_encoder = BgeQueryEncoder(package + model_name)

run_latency_datasets()

100%|██████████| 57638/57638 [00:00<00:00, 1467801.40it/s]


4.76 s +- 282 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 7.3457 ms. Experiment details: 4.76 s +- 282 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

143 ms +- 4.76 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Document retrieval time: Latency per query: 0.2207 ms. Experiment details: 143 ms +- 4.76 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)



100%|██████████| 3633/3633 [00:00<00:00, 880142.46it/s]


1.64 s +- 63.8 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 5.0774 ms. Experiment details: 1.64 s +- 63.8 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

5.43 ms +- 232 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0168 ms. Experiment details: 5.43 ms +- 232 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



100%|██████████| 5183/5183 [00:00<00:00, 772862.54it/s]


3.84 s +- 90.4 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 12.8 ms. Experiment details: 3.84 s +- 90.4 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

8.04 ms +- 597 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0268 ms. Experiment details: 8.04 ms +- 597 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



In [10]:
from encoders.gte_base_en_encoder import GTEBaseEncoder

package = "Alibaba-NLP/"
model_name = "gte-base-en-v1.5"
q_encoder = GTEBaseEncoder(package + model_name)
model_directory = 'gte_base_en_v1_5'
pipeline_name = "BM25 >> " + model_name

run_latency_datasets()

100%|██████████| 57638/57638 [00:00<00:00, 1392552.51it/s]


16.3 s +- 546 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 25.1543 ms. Experiment details: 16.3 s +- 546 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

158 ms +- 3.5 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Document retrieval time: Latency per query: 0.2438 ms. Experiment details: 158 ms +- 3.5 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)



100%|██████████| 3633/3633 [00:00<00:00, 1348367.97it/s]


4.28 s +- 305 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 13.2508 ms. Experiment details: 4.28 s +- 305 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

6.18 ms +- 17.5 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0191 ms. Experiment details: 6.18 ms +- 17.5 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



100%|██████████| 5183/5183 [00:00<00:00, 1745549.83it/s]


11.9 s +- 153 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 39.6667 ms. Experiment details: 11.9 s +- 153 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

9.77 ms +- 24.6 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0326 ms. Experiment details: 9.77 ms +- 24.6 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



In [8]:
from encoders.snowflake_arctic_embed_m import SnowFlakeQueryEncoder

package = "Snowflake/"
model_name = "snowflake-arctic-embed-m"
q_encoder = SnowFlakeQueryEncoder(package + model_name)
model_directory = 'snowflake'
pipeline_name = "BM25 >> " + model_name

run_latency_datasets()

100%|██████████| 57638/57638 [00:00<00:00, 1299353.92it/s]


18.9 s +- 461 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 29.1667 ms. Experiment details: 18.9 s +- 461 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

151 ms +- 3.8 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Document retrieval time: Latency per query: 0.233 ms. Experiment details: 151 ms +- 3.8 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)



100%|██████████| 3633/3633 [00:00<00:00, 1643787.10it/s]


6.06 s +- 168 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 18.7616 ms. Experiment details: 6.06 s +- 168 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

6.95 ms +- 349 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0215 ms. Experiment details: 6.95 ms +- 349 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



100%|██████████| 5183/5183 [00:00<00:00, 1390588.99it/s]


13.1 s +- 523 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 43.6667 ms. Experiment details: 13.1 s +- 523 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

9.74 ms +- 43.8 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0325 ms. Experiment details: 9.74 ms +- 43.8 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



In [12]:
from encoders.snowflake_arctic_embed_m import SnowFlakeQueryEncoder

package = "Snowflake/"
model_name = "snowflake-arctic-embed-xs"
model_directory = 'snowflake'
q_encoder = SnowFlakeQueryEncoder(package + model_name)
pipeline_name = "BM25 >> " + model_name

for dataset_name in dataset_list:
    measure_latency(dataset_name)

100%|██████████| 57638/57638 [00:00<00:00, 1351743.94it/s]


2.28 s +- 80.3 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 3.5185 ms. Experiment details: 2.28 s +- 80.3 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

130 ms +- 1.87 ms per loop (mean +- std. dev. of 7 runs, 10 loops each)

Document retrieval time: Latency per query: 0.2006 ms. Experiment details: 130 ms +- 1.87 ms per loop (mean +- std. dev. of 7 runs, 10 loops each)



100%|██████████| 3633/3633 [00:00<00:00, 1597264.83it/s]


691 ms +- 15.4 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 2.1393 ms. Experiment details: 691 ms +- 15.4 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

4.7 ms +- 43 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0146 ms. Experiment details: 4.7 ms +- 43 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



100%|██████████| 5183/5183 [00:00<00:00, 1235525.87it/s]


1.65 s +- 24.9 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 5.5 ms. Experiment details: 1.65 s +- 24.9 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

7.55 ms +- 17.4 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0252 ms. Experiment details: 7.55 ms +- 17.4 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



In [13]:
from fast_forward.encoder import TCTColBERTQueryEncoder

package = "castorini/"
model_name = "tct_colbert-msmarco"
q_encoder = TCTColBERTQueryEncoder(package + model_name)
model_directory = 'tct_colbert'
pipeline_name = "BM25 >> " + 'tct_colbert_msmarco'

run_latency_datasets()

/home/anistor/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 57638/57638 [00:00<00:00, 1376990.25it/s]


20.3 s +- 725 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 31.3272 ms. Experiment details: 20.3 s +- 725 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

203 ms +- 20.5 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Document retrieval time: Latency per query: 0.3133 ms. Experiment details: 203 ms +- 20.5 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)



100%|██████████| 3633/3633 [00:00<00:00, 1476827.53it/s]


8.09 s +- 177 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 25.0464 ms. Experiment details: 8.09 s +- 177 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

6.62 ms +- 333 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0205 ms. Experiment details: 6.62 ms +- 333 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



100%|██████████| 5183/5183 [00:00<00:00, 1523411.19it/s]


7.84 s +- 128 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 26.1333 ms. Experiment details: 7.84 s +- 128 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

9.73 ms +- 583 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0324 ms. Experiment details: 9.73 ms +- 583 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



In [14]:
from encoders.e5 import E5QueryEncoder

package = "intfloat/"
model_name = "e5-base-v2"
model_directory = 'e5'
q_encoder = E5QueryEncoder(package + model_name)
pipeline_name = "BM25 >> " + model_name

run_latency_datasets()

100%|██████████| 57638/57638 [00:00<00:00, 1392111.47it/s]


12.7 s +- 217 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 19.5988 ms. Experiment details: 12.7 s +- 217 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

166 ms +- 3.79 ms per loop (mean +- std. dev. of 7 runs, 10 loops each)

Document retrieval time: Latency per query: 0.2562 ms. Experiment details: 166 ms +- 3.79 ms per loop (mean +- std. dev. of 7 runs, 10 loops each)



100%|██████████| 3633/3633 [00:00<00:00, 1505870.78it/s]


3.47 s +- 84.4 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 10.743 ms. Experiment details: 3.47 s +- 84.4 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

6.28 ms +- 63.2 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0194 ms. Experiment details: 6.28 ms +- 63.2 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



100%|██████████| 5183/5183 [00:00<00:00, 1321604.82it/s]


9.82 s +- 660 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 32.7333 ms. Experiment details: 9.82 s +- 660 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

9.97 ms +- 283 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0332 ms. Experiment details: 9.97 ms +- 283 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



In [15]:
from encoders.e5 import E5QueryEncoder

model_name = "e5-small-v2"
q_encoder = E5QueryEncoder(package + model_name)
pipeline_name = "BM25 >> " + model_name

run_latency_datasets()

100%|██████████| 57638/57638 [00:00<00:00, 357079.88it/s]


3.7 s +- 43.3 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 5.7099 ms. Experiment details: 3.7 s +- 43.3 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

121 ms +- 218 us per loop (mean +- std. dev. of 7 runs, 10 loops each)

Document retrieval time: Latency per query: 0.1867 ms. Experiment details: 121 ms +- 218 us per loop (mean +- std. dev. of 7 runs, 10 loops each)



100%|██████████| 3633/3633 [00:00<00:00, 1524858.04it/s]


1.01 s +- 7.79 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 3.1269 ms. Experiment details: 1.01 s +- 7.79 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

4.91 ms +- 144 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0152 ms. Experiment details: 4.91 ms +- 144 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



100%|██████████| 5183/5183 [00:00<00:00, 1579415.70it/s]


2.88 s +- 48.5 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 9.6 ms. Experiment details: 2.88 s +- 48.5 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

7.58 ms +- 11.1 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0253 ms. Experiment details: 7.58 ms +- 11.1 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



In [16]:
from encoders.e5 import E5QueryEncoder

model_name = "e5-base-unsupervised"
q_encoder = E5QueryEncoder(package + model_name)
pipeline_name = "BM25 >> " + model_name

run_latency_datasets()

100%|██████████| 57638/57638 [00:00<00:00, 1325754.29it/s]


13 s +- 1.88 s per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 20.0617 ms. Experiment details: 13 s +- 1.88 s per loop (mean +- std. dev. of 7 runs, 1 loop each)

166 ms +- 5.56 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Document retrieval time: Latency per query: 0.2562 ms. Experiment details: 166 ms +- 5.56 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)



100%|██████████| 3633/3633 [00:00<00:00, 1495378.45it/s]


4.3 s +- 66 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 13.3127 ms. Experiment details: 4.3 s +- 66 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

6.64 ms +- 121 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0206 ms. Experiment details: 6.64 ms +- 121 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



100%|██████████| 5183/5183 [00:00<00:00, 1425700.26it/s]


10.9 s +- 242 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 36.3333 ms. Experiment details: 10.9 s +- 242 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

9.86 ms +- 59 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0329 ms. Experiment details: 9.86 ms +- 59 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



In [17]:
from encoders.nomic import NomicQueryEncoder

package = "nomic-ai/"
model_name = "nomic-embed-text-v1"
q_encoder = NomicQueryEncoder(package + model_name)
model_directory = 'nomic'
pipeline_name = "BM25 >> " + model_name

run_latency_datasets()

<All keys matched successfully>
100%|██████████| 57638/57638 [00:00<00:00, 1227369.53it/s]


21.8 s +- 342 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 33.642 ms. Experiment details: 21.8 s +- 342 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

162 ms +- 3.33 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Document retrieval time: Latency per query: 0.25 ms. Experiment details: 162 ms +- 3.33 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)



100%|██████████| 3633/3633 [00:00<00:00, 1108856.53it/s]


6.29 s +- 27.5 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 19.4737 ms. Experiment details: 6.29 s +- 27.5 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

6.57 ms +- 43.7 us per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0203 ms. Experiment details: 6.57 ms +- 43.7 us per loop (mean +- std. dev. of 7 runs, 100 loops each)



100%|██████████| 5183/5183 [00:00<00:00, 1352184.96it/s]


16.1 s +- 511 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

Query encoding time: Latency per query: 53.6667 ms. Experiment details: 16.1 s +- 511 ms per loop (mean +- std. dev. of 7 runs, 1 loop each)

12.2 ms +- 1.84 ms per loop (mean +- std. dev. of 7 runs, 100 loops each)

Document retrieval time: Latency per query: 0.0407 ms. Experiment details: 12.2 ms +- 1.84 ms per loop (mean +- std. dev. of 7 runs, 100 loops each)

